In [38]:
import pandas as pd
import re
import datetime as dt
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import datetime
#import mysql.connector
from pandas.io import sql
import sqlalchemy
import re
import psycopg2
from sqlalchemy import create_engine
#from baseball_scraper import playerid_lookup
from pybaseball import playerid_lookup
from urllib.parse import quote_plus

In [30]:
# %run elon_functions.py

# Give the location of the file 
loc = ("data/GameLogs_1419.csv") 
# To open Workbook 
df = pd.read_csv(loc)


print(len(df))
to_delete = []
#delete the double headers. Confuses the model.
for row, col in df.iterrows():
    r = df.iloc[row]
    if (len(r["Date"]) >10):
        to_delete.append(row)
df = df.drop(to_delete, axis = 0)      
print("deleted double header rows")
df.reset_index(inplace = True, drop = True)

170705
deleted double header rows


In [4]:
df["Date"] = [x[0:10] for x in df["Date"]]
df['Date'] = df['Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))

In [5]:
def make_player_id_dict(df):
    player_names = list(df["Player"].unique())
    name_id_dict = {}
    for x in player_names:
        first_name = re.findall('(.*) ', x)[0]
        last_name = re.findall('.* (.*)\\\\', x)[0]
        
        if (first_name[1] =='.'):
            first_name = first_name[:2] + ' ' + first_name[2:]
        lookup_id = 0
        try:
            lookup_id_df = playerid_lookup(last_name, first_name)
            if(len(lookup_id_df) > 1):
                lookup_id_df = lookup_id_df.dropna()
                if(len(lookup_id_df) >1):
                    lookup_id = -1 # player has two people with the same name
                else:
                    lookup_id = lookup_id_df['key_mlbam'].values[0]
            else:
                lookup_id = lookup_id_df['key_mlbam'].values[0]
        except:
            lookup_id = -1
        if (lookup_id != -1):
            name_id_dict[(first_name, last_name)] = lookup_id
    
    return name_id_dict

my_dict = make_player_id_dict(df)

Gathering player lookup table. This may take a moment.


In [6]:
def append_next_game_details(df2):
    #determine if the player gets a hit the next game
    df_huge = pd.DataFrame()
    player_names= list(df2["Name"].unique())
    for name in player_names:
        df_player = df2[df2["Name"] == name]
        df_p = df_player.sort_values(by = ["fixed_date"])
        df_p["next_game_BA"] = df_p["BA"].shift(-1)
        df_p["next_game_Name"] = df_p["Name"].shift(-1)
        df_p["next_game_Date"] = df_p["fixed_date"].shift(-1)
        df_p["key"] = df_p["key"].shift(-1)
        df_huge = pd.concat([df_huge, df_p], axis = 0)
    return df_huge    

In [7]:
#dumping the contents of my_dict into a picklized file
import pickle
with open('data/mydict.pickle','wb') as handle:
    pickle.dump(my_dict, handle)

In [8]:
list(my_dict)[-1]

('Brendan', 'McKay')

# Once the scraping is complete, we then combine our two dataframes

In [9]:
#loc = ("data/aggregates.csv") 
loc = ("data/aggregates2.csv")
# To open Workbook 
df2 = pd.read_csv(loc)

In [14]:
dates = []
for i,x in enumerate(df2["Game_date"]):
    try:
        j = dt.datetime.strptime(x,'%m/%d/%Y')
    except:
        try:
            #j = dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
            j = dt.datetime.strptime(x,'%Y-%m-%d')
        except:
            try:
                j = dt.datetime.strptime(x,'%m/%d/%Y %H:%M')
            except:
                print(i,x, "didn't pass third case")
    dates.append(j)

In [15]:
assert(len(dates) == len(df2))
df2["fixed_date"] = [str(k)[:10] for k in dates]

Create a hash key with the player name and date information in order to merge df and df2

In [16]:
name_date = []
#make the keys for the game log df
for row,col in df.iterrows():
    row = df.iloc[row]
    player_name = row["Player"]
    first_name = re.findall('(.*) ', player_name)[0]
    #if the name has a '.' in it like "A.J."
    if (first_name[1] =='.'):
        first_name = first_name[:2] + ' ' + first_name[2:]
    last_name = re.findall('.* (.*)\\\\', player_name)[0]
    name = first_name + ' ' + last_name
    date = str(row["Date"])[:10]
    name_date.append(hash(name+date))
df["key"] = name_date

In [17]:
name_date = []
#make the key for the aggregate stats df
for row,col in df2.iterrows():
    row = df2.iloc[row]
    player_name = row["Name"]
    date = row["fixed_date"]
#     print(player_name+date)
    name_date.append(hash(player_name+date))
#create a key that will enable us to merge dataframes. 
df2["key"] = name_date

In [18]:
df2 = append_next_game_details(df2)

In [602]:
# dic = {}
# duplicates = []
# for j in df["key"]:
#     if (j in dic):
#         duplicates.append(j)
#     else:
#         dic[j] = 1
# print(duplicates)

In [19]:
#joins are only possible if the keys and their datatypes are the same
df2["key"] =df2["key"].astype(float)
df["key"] = df["key"].astype(float)

In [20]:
mega_df = df2.join(df.set_index('key'), on='key', rsuffix='agg')

In [21]:
mega_df[mega_df["Pos.Summary"].notnull()]

,BA,BABIP,BIP,Game_date,Games_played_to_date,ISO,LA_avg,LA_median,Name,OBP,...,X3B,HR,RBI,BB,SO,WPA,RE24,aLI,BOP,Pos.Summary
1,0.000000,0.000000,0.500000,2014-04-02,2,0.000000,NaN,NaN,Travis d'Arnaud,0.142857,...,0.0,0.0,0.0,1.0,2.0,-0.037,-0.425,0.870,7.0,C
4,0.000000,0.000000,0.600000,2014-04-06,5,0.000000,NaN,NaN,Travis d'Arnaud,0.117647,...,0.0,0.0,0.0,0.0,0.0,0.066,0.277,0.638,7.0,C
5,0.105263,0.153846,0.684211,2014-04-08,6,0.052632,NaN,NaN,Travis d'Arnaud,0.190476,...,0.0,0.0,1.0,0.0,0.0,0.028,0.453,1.220,7.0,C
6,0.130435,0.176471,0.739130,2014-04-09,7,0.043478,NaN,NaN,Travis d'Arnaud,0.200000,...,0.0,0.0,0.0,0.0,0.0,-0.186,-1.535,1.913,7.0,C
10,0.162162,0.178571,0.763158,2014-04-14,11,0.100996,NaN,NaN,Travis d'Arnaud,0.219512,...,0.0,0.0,0.0,2.0,1.0,-0.005,0.259,0.180,7.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33898,0.270522,0.295405,0.860037,2014-09-17,136,0.121743,NaN,NaN,Ben Zobrist,0.351525,...,0.0,0.0,0.0,0.0,0.0,-0.120,-0.846,1.153,1.0,LF
33899,0.270370,0.295011,0.861060,2014-09-19,137,0.120854,NaN,NaN,Ben Zobrist,0.350877,...,0.0,0.0,1.0,1.0,0.0,0.060,1.147,0.620,1.0,LF
33902,0.275046,0.300000,0.863309,2014-09-23,140,0.120638,NaN,NaN,Ben Zobrist,0.354788,...,0.0,0.0,1.0,0.0,0.0,0.004,0.773,0.432,1.0,DH
33903,0.276173,0.301053,0.864528,2014-09-24,141,0.121331,NaN,NaN,Ben Zobrist,0.355140,...,0.0,0.0,1.0,0.0,0.0,0.033,-0.079,0.820,1.0,SS


In [22]:
bad_cols = ['160421_183314_percentage_hits', '160421_204540_percentage_hits',
       '160421_194503_percentage_hits', '160421_182611_percentage_hits',
       '160421_195036_percentage_hits', '160421_191133_percentage_hits',
       '160421_200605_percentage_hits', '160421_202513_percentage_hits',
       '160421_193312_percentage_hits', '160421_195422_percentage_hits',
       '160421_182835_percentage_hits','160421_182316_percentage_hits', '160421_185447_percentage_hits']

In [23]:
mega_df = mega_df.drop(columns = ["Unnamed: 0", "Game_date","fixed_date"] + bad_cols)

KeyError: "['160421_183314_percentage_hits', '160421_204540_percentage_hits', '160421_194503_percentage_hits', '160421_182611_percentage_hits', '160421_195036_percentage_hits', '160421_191133_percentage_hits', '160421_200605_percentage_hits', '160421_193312_percentage_hits', '160421_195422_percentage_hits', '160421_182835_percentage_hits', '160421_182316_percentage_hits', '160421_185447_percentage_hits'] not found in axis"

In [24]:
mega_df.columns

Index(['BA', 'BABIP', 'BIP', 'Game_date', 'Games_played_to_date', 'ISO',
       'LA_avg', 'LA_median', 'Name', 'OBP',
       ...
       'X3B', 'HR', 'RBI', 'BB', 'SO', 'WPA', 'RE24', 'aLI', 'BOP',
       'Pos.Summary'],
      dtype='object', length=113)

In [25]:
mega_df = mega_df.reindex(columns=['Name', 'Tm', 'BA', 'BABIP', 'BIP', 'Games_played_to_date', 'ISO', 'LA_avg', 'LA_median','OBP', 'OPS', 'PA', 'SLG', 'Walks', 'mlbam_code',
       'FT_percentage_hits', 'pHitsByZone5', 'Weak_lsa_p', 'Topped_lsa_p',
       'Under_lsa_p', 'Flare/Burner_lsa_p', 'SolidContact_lsa_p',
       'Barrel_lsa_p', 'FF_percentage_hits', 'FC_percentage_hits',
       'SI_percentage_hits', 'pHitsByZone2', 'pHitsByZone4', 'pHitsByZone14',
       'CH_percentage_hits', 'SL_percentage_hits', 'pHitsByZone1',
       'pHitsByZone13', 'pHitsByZone6', 'KC_percentage_hits', 'pHitsByZone7',
       'pHitsByZone9', 'CU_percentage_hits', 'FS_percentage_hits',
       'pHitsByZone8', 'pHitsByZone12', 'pHitsByZone3', 'pHitsByZone11',
       'FO_percentage_hits', 'KN_percentage_hits', 'FA_percentage_hits',
       'EP_percentage_hits', 'SC_percentage_hits', 'key', 'next_game_BA',
       'next_game_Name', 'next_game_Date', 'next_game_Key', 'Player', 'Date','Opp', 'PAagg', 'AB', 'R', 'H', 'X2B', 'X3B', 'HR', 'RBI', 'BB',
       'SO', 'WPA', 'RE24', 'aLI', 'BOP', 'Pos.Summary'])

In [26]:
for col in mega_df.columns:
    if (mega_df[col].dtype =='float64'):
        mega_df[col] = mega_df[col].round(5)

In [27]:
mega_df = mega_df[mega_df['BA'].notna()]

In [28]:
mega_df= mega_df.reset_index()

In [29]:
mega_df.to_csv('data/aggregate_stats_v2.csv')

In [39]:
database_username = 'raguilar'
database_password = quote_plus("Lb$bYdu^2P%gv%D6cfqNJ7@aAhE&jT7r")
database_ip       = 'localhost'
database_name     = 'mlb_dw'

database_connection = sqlalchemy.create_engine('postgresql+psycopg2://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_timeout = 28800, max_overflow = -1)
connection = database_connection.connect()
print("made connection")
# try:
num = 0
try:
    while(num < (len(mega_df))):
        ie = 'append'
        if (num == 0):
            ie = 'replace'
        val = min((len(mega_df) -num), 6000)
        print(num,val)
        mega_df[num: num+ val].to_sql('aggregate_stats_v1', connection,index=True, if_exists=ie, chunksize = 25000)
        print("table "+ ie + " successful")
        num +=val
except:
    print("error")
    connection.close()
connection.close()

made connection
0 6000
table replace successful
6000 6000
table append successful
12000 6000
table append successful
18000 6000
table append successful
24000 6000
table append successful
30000 4022
table append successful


In [337]:
import pickle
my_dict ={}
with open('data/mydict.pickle','rb') as handle:
    my_dict = pickle.dumps(handle)